# Problema 3.

Considere que, apoyándose en el desarrollo de la Ciencia de Datos y la Optimización avanzada, usted desea
conformar un nuevo equipo de fútbol. Para esto, usted ha logrado recopilar información sobre 325 futbolistas.
Esta información corresponde a la valoración que se tiene, para cada uno de ellos, de las 4 posiciones en
el campo de juego: Arquero, Defensa, Mediocampista y Delantero; y al precio de contratar a ese jugador.
Esta valoración está medida en un coeficiente que va desde el 0 al 100, donde el 100 es la mayor valoración.
Además, cuanta con la información de qué jugadores no debeŕıan compartir equipo con qué jugadores. Esta
información está clasificada como 0 (pueden compartir equipo), 1 (conflicto 1: preferentemente no debeŕıan
compartir equipo) y 2 (conflicto 2: no debeŕıan compartir equipo). Toda esta información se encuentra en el
archivo Datos.xls, en la hojas Caracteŕısticas y Conflictos.

Basado en esta información, se le pide conformar un equipo con la mayor valoración posible que cuente con
33 jugadores (no se puede contratar a un jugador en más de una posición), de los cuales 3 son Arqueros, 10
Defensas, 12 son Mediocampistas y 8 son Delanteros, y que se limite por el presupuesto disponible, que es de
22000 pesos. Para esto, usted sabe que cada jugador de la lista puede ser contratado en cualquier de las 4
posiciones, que la valoración total del equipo es la suma de las valoraciones individuales de la posición escogida
para cada jugador contratado, y que el costo total de la plantilla de jugadores es la suma de los precios de
contratación de todos los jugadores contratados.

*(a) (10 puntos) Diseñe un algoritmo, prográmelo y ejecútelo en Python, para seleccionar aleatoriamente jugadores a cada una de las 33 posiciones requeridas sin sobrepasar el presupuesto disponible. Registre la solución en: jugadores seleccionados y su posición, valoración total del equipo, total de conflictos (separados en conflictos 1 y conflictos 2) y listados de casos (jugadores en cada conflicto), y el costo total de la plantilla. Una vez diseñado este algoritmo, ejecútelo 1000 veces, y entregue la solución que tiene menos puntaje en conflictos (sume 1 punto a cada conflicto 1 y 5 puntos a cada conflicto 2). En caso de que haya más de una solución con el mismo puntaje en conflictos, escoja aquella que tenga mayor valoración total.*

In [13]:
from gurobipy import *
import pandas as pd

In [14]:
modelo = Model("modelo_futbol")

In [16]:
## Definimos variables 
# Número de jugadores
num_jugadores = 325

# Número de posiciones
num_posiciones = 4

# Número de jugadores por posición
num_arqueros = 3
num_defensas = 10
num_mediocampistas = 12
num_delanteros = 8

# Presupuesto disponible
presupuesto = 22000

datos = pd.read_excel("Datos.xlsx", sheet_name="Características")


In [23]:
#Variables de desición definidas
var_desicion = {}
for i in range(num_jugadores):
    for j in ["Arquero", "Defensa", "Mediocampista", "Delantero"]:
        var_desicion[i, j] = modelo.addVar(vtype=GRB.BINARY, lb=0, name=f"x-{i}-{j}")
#Funcion Objetivo
modelo.setObjective(
    quicksum(
        datos.loc[i, "Arquero"] * var_desicion[i, "Arquero"] +
        datos.loc[i, "Defensa"] * var_desicion[i, "Defensa"] +
        datos.loc[i, "Mediocampista"] * var_desicion[i, "Mediocampista"] +
        datos.loc[i, "Delantero"] * var_desicion[i, "Delantero"]
        for i in range(num_jugadores)
    ),
    GRB.MAXIMIZE
)

#Restricciones

modelo.addConstrs(
    quicksum(var_desicion[i, "Arquero"] for i in range(num_jugadores)) == num_arqueros
    for j in range(num_posiciones)
)
modelo.addConstrs(
    quicksum(var_desicion[i, "Defensa"] for i in range(num_jugadores)) == num_defensas
    for j in range(num_posiciones)
)
modelo.addConstrs(
    quicksum(var_desicion[i, "Mediocampista"] for i in range(num_jugadores)) == num_mediocampistas
    for j in range(num_posiciones)
)
modelo.addConstrs(
    quicksum(var_desicion[i, "Delantero"] for i in range(num_jugadores)) == num_delanteros
    for j in range(num_posiciones)
)
# Optimizar el modelo
modelo.optimize()





{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>}

In [24]:
#Respuesta óptima
modelo.optimize()

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (linux64)

CPU model: AMD EPYC 7763 64-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads



GurobiError: Model too large for size-limited license; visit https://www.gurobi.com/free-trial for a full license


*(b) (10 puntos) Diseñe un modelo de optimización lineal en variables binarias, prográmelo y resuélvalo en Python/GUROBI, para encontrar la conformación de la plantilla de jugadores (33 posiciones), garantizando que no existan conflictos y que no se sobrepasa el presupuesto disponible. Compare esta solución con la obtenida en la parte (a).*